# Carga del *Breast Cancer Wisconsin Data Set*

Comenzamos realizando la carga del *Breast Cancer Wisonsin Data Set* que determina la presencia o no de tejido canceroso en función de una serie de muestras extraídas de un conjunto de 699 pacientes. Utilizaremos ``pandas`` y su función ``read_csv()`` para realizar la carga del fichero.

In [1]:
import pandas as pd
import numpy as np

col_names = ['sample_code', 'clump_thickness', 'uniformity_cell_size', 
            'uniformity_cell_shape', 'marginal_adhesion', 'single_epithelial_cell_size',
            'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class']
df = pd.read_csv('breast-cancer-wisconsin.csv', names = col_names)
print(df.shape)
df.head()

(699, 11)


,sample_code,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


# Preprocesado y búsqueda de los parámetros óptimos mediante ``GridSearchCV``.

A continuación vamos a realizar un sencillo preprocesado del *DataFrame* obtenido antes de realizar una búsqueda en *grid* de aquellos hiperparámetros que optimicen el proceso de clasificación. Comenzamos eliminado de nuestro *DataFrame* la columna ``sample_code`` pues tan sólo es una etiqueta identificativa de cada muestra y no aporta información relevante con respecto a la tarea de clasificación.

In [2]:
df.drop('sample_code', inplace = True, axis = 1)
df.head()

,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


Comprobamos que efectivamente, en este conjunto de datos existen valores nulos, en concreto, en la columna ``bare_nuclei``. Si bien podríamos eliminar las filas correspondientes a esos valores faltantes. Utilizaremos una estrategia de imputado de valores. 

In [3]:
df[df == '?'] = np.nan
df.isnull().sum()

clump_thickness                 0
uniformity_cell_size            0
uniformity_cell_shape           0
marginal_adhesion               0
single_epithelial_cell_size     0
bare_nuclei                    16
bland_chromatin                 0
normal_nucleoli                 0
mitoses                         0
class                           0
dtype: int64

Por otra parte, realizamos un conteo de cuántos elementos existen en cada clase. En este caso nos encontramos frente a un problema de clasificación binaria con clases "2" (tejido benigno) y "4" (tejido canceroso). Además, observamos cierto desequilibrio entre estas clases, siendo aproximadamente un 65.5% muestras de  clase "2" y 34.5% de la clase "4". Esto condicionará el uso de una métrica u otra a la hora de determinar el rendimiento de nuestros clasificadores.

In [4]:
df['class'].value_counts()

2    458
4    241
Name: class, dtype: int64

Como se ha mencionado previamente, vamos a hacer una búsqueda en *grid* de los parámetros óptimos para este problema de clasificación. En concreto, vamos a realizar las siguientes tareas:

- Creación de un ``Pipeline`` de trabajo con:    
    - Imputado de valores faltantes con ``SimpleImputer`` con la estrategia ``most_frequent``.
    - Escalado de los datos en el rango [0, 1] mediante ``MinMaxScaler``.
    - Inicalización de un clasificador ``MLPClassifier``.
        

- Creación de un espacio de búsqueda para los siguientes hiperparámetros:
    - Número de iteraciones máximas ``max_iter``.
    - Función de activación ``activation``.
    - Tasa de aprendizaje inicial ``learning_rate_init``.
    - Tamaño y número de capas ocultas ``hidden_layer_sizes``.
    - Algoritmo de optimización de los pesos ``solver``.

Además, al tratar con un data set ligeramente desequilibrado, haremos uso del ``balanced_accuracy_score`` como métrica del rendimiento de las redes neuronales.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from random import seed
from time import time

# Incializamos semilla aleatoria
seed = seed(time())
# Separamos en variables de entrada y variable objetivo
X = df[df.columns[:-1]]
Y = df[df.columns[-1]]

# Creamos el pipeline de trabajo
pipe = Pipeline(steps = [('num_imputer', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),
                ('scaler', MinMaxScaler()),
                ('MLPClassifier', MLPClassifier(random_state = seed))]
                )
# Creamos el espacio de búsqueda de hiperparámetros
aux = [10, 100]
space = [{'MLPClassifier__solver': ['lbfgs', 'adam'], 
        'MLPClassifier__max_iter': [500, 1500, 2000, 2500, 3000],
        'MLPClassifier__activation': ['relu', 'logistic', 'tanh', 'identity'],
        'MLPClassifier__learning_rate_init': [0.0001, 0.001, 0.01, 0.1],
        'MLPClassifier__hidden_layer_sizes': [(x, ) for x in aux] + [(x, y) for x in aux for y in aux]}
        ]

cross_val = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = seed)
grid_search = GridSearchCV(estimator = pipe, param_grid = space,
                            cv = cross_val, return_train_score = True,
                            n_jobs = -1, scoring = 'balanced_accuracy')

res = grid_search.fit(X, Y)

In [9]:
res.scorer_

make_scorer(balanced_accuracy_score)

In [6]:
res.best_estimator_

Pipeline(steps=[('num_imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', MinMaxScaler()),
                ('MLPClassifier',
                 MLPClassifier(activation='logistic',
                               hidden_layer_sizes=(10, 10), max_iter=1500))])

In [7]:
res.best_score_

0.970170692431562

In [8]:
res.best_params_

{'MLPClassifier__activation': 'logistic',
 'MLPClassifier__hidden_layer_sizes': (10, 10),
 'MLPClassifier__learning_rate_init': 0.001,
 'MLPClassifier__max_iter': 1500,
 'MLPClassifier__solver': 'adam'}

De los resultados anteriores vemos que se obtiene la mejor *balanced accuracy* (~97.02%) con los siguientes parámetros:

- Función de activación ``logistic``.
- Topología de la red neuronal con dos capas ocultas y 10 neuronas por capa ``(10, 10)``.
- Tasa de aprendizaje 0.001.
- Iterciones máximas 1500.
- Algoritmo de optimización ``adam``.